In [ ]:
import sys
sys.path.insert(1, '/home/sbanik@quansight.com/demo-dashboards')

from intake_utils import (
    catalog_init,
    list_catalog,
    view_catalog)

In [ ]:
catalog = catalog_init()

In [ ]:
list(catalog)

In [ ]:
data = catalog.flu_pneumonia_covid.read()

### Converting json to pandas dataframe

In [ ]:
import pandas as pd
df = pd.json_normalize(data)

In [ ]:
df.head()

### Checking datatypes

In [ ]:
df.dtypes

In [ ]:
df.info()

### Updating datatypes

In [ ]:
df = df.astype({'data_as_of': 'datetime64[ns]', 
                'start_week': 'datetime64[ns]',
                'end_week': 'datetime64[ns]',
                'mmwryear': 'Int64',
                'mmwrweek': 'Int64',
                'week_ending_date': 'datetime64[ns]',
                'covid_19_deaths': 'Int64',
                'total_deaths': 'Int64',
                'pneumonia_deaths': 'Int64',
                'influenza_deaths': 'Int64',
                'pneumonia_or_influenza': 'Int64',
                'pneumonia_influenza_or_covid': 'Int64'
               })

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
import numpy as np

df = df.replace(np.nan, 0)

In [ ]:
sorted_df = df.sort_values(by=['total_deaths'], ascending=False)


In [ ]:
sorted_df.head(10)

In [ ]:
group_df = df.groupby(['age_group']).total_deaths.sum().reset_index()

In [ ]:
group_df['percentage_increase'] = group_df.total_deaths.pct_change().replace(np.nan, 0)

In [ ]:
group_df

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import hvplot.pandas
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', 'vega')

### Total deaths based on age-group

In [ ]:
bar_chart = hv.Bars(pd.DataFrame(group_df, columns=['age_group','total_deaths']), label="Total deaths based on age-group") 
bar_chart.opts(xrotation=45, width=600, height=400) 

In [ ]:
curve= hv.Curve((group_df.age_group, group_df.percentage_increase),"Age-group","percentage increase", label=('Percentage increase in death'))
curve.opts(width=600, height=400) 

#### Insight
- With increase in age-group, the overall death count increased substantially 
- maximum increase could be observed between age group *0-17* to *18-64*

#### Based on jurisdiction

In [ ]:
import altair as alt

In [ ]:
chart = alt.Chart(sorted_df).mark_circle(size=90).encode(
    x='covid_19_deaths',
    y='influenza_deaths',
    color='jurisdiction',
    tooltip=['total_deaths', 'jurisdiction', 'covid_19_deaths']
).interactive()

altair_pane = pn.panel(chart)
altair_pane

#### Insight
 - United states has the highest deseased count due to influenza and covid
 - Deaths due to influenza and covid shows a certain correlation based on the count 
 - Alaska has zero deseased count 